# Gapfilling with Sco Model

In [1]:
import cobra
from cobra.core import Metabolite, Reaction
from cobra.io import read_sbml_model, load_json_model
from cobra.flux_analysis import gapfill
import os, re
import pandas as pd

print(os.getcwd())

ext_dir = '/../../../data/external'
phenomics = '/../../phenomics'

# Import model
# model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM.xml")
model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM-Updated.xml")

# Mapping of the sco_model reaction model
sco_model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Sco-GEM.xml")

/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/strep/Code/streptsd/notebooks/gapfilling/1.gapfill_w_sco


'' is not a valid SBML 'SId'.


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-13


# 1. Get all untestable and false negative compounds 

In [10]:
# Import agreed biolog vs model data
agreed_bio_data = pd.read_csv(f"{os.getcwd()}{phenomics}/agreed_bio_data_Salb.csv")
agreed_bio_data['model_simulation_0.05'] = agreed_bio_data['model_simulation_0.05'].astype('boolean')

# # Mapping between bigg and metax names
# bigg_mnmx = pd.read_json(f"{os.getcwd()}/{ext_dir}/metanetx.json")

# # Mapping files for bigg compound to all other names
# name_map = pd.read_csv(f"{os.getcwd()}/{ext_dir}/biggmodels_metabolites.txt",sep='\t')
# name_map = name_map.drop(['bigg_id', 'model_list', 'old_bigg_ids'], axis=1)

agreed_bio_data

,index,activity,chemical,category,moa,bigg,exchange,model_simulation_0.05,model_simulation_0.1,model_simulation_0.2
0,PM01_A01,False,Negative Control,carbon,"C-Source, negative control",#,#,<NA>,NaN,NaN
1,PM01_A02,True,L-Arabinose,carbon,"C-Source, carbohydrate",arab__L,EX_arab__L_e,False,False,False
2,PM01_A03,True,N-Acetyl-D-Glucosamine,carbon,"C-Source, carbohydrate",acgam,EX_acgam_e,<NA>,NaN,NaN
3,PM01_A04,False,D-Saccharic acid,carbon,"C-Source, carboxylic acid",glcr,EX_glcr_e,<NA>,NaN,NaN
4,PM01_A05,True,Succinic acid,carbon,"C-Source, carboxylic acid",succ,EX_succ_e,True,True,True
...,...,...,...,...,...,...,...,...,...,...
379,PM04_H08,False,p-Aminobenzene Sulfonic acid,phosphate & sulphur,"S-Source, organic",4abz,EX_4abz_e,<NA>,NaN,NaN
380,PM04_H09,False,Butane Sulfonic acid,phosphate & sulphur,"S-Source, organic",butso3,EX_butso3_e,<NA>,NaN,NaN
381,PM04_H10,False,2-Hydroxyethane Sulfonic acid,phosphate & sulphur,"S-Source, organic",isetac,EX_isetac_e,<NA>,NaN,NaN
382,PM04_H11,False,Methane Sulfonic acid,phosphate & sulphur,"S-Source, organic",mso3,EX_mso3_e,<NA>,NaN,NaN


In [11]:
print(f"\nFalse negative (actual true, predicted false):\n")
agreed_bio_data_nan_drop = agreed_bio_data[agreed_bio_data['model_simulation_0.05'].notna()]
false_negatives = agreed_bio_data_nan_drop[
    agreed_bio_data_nan_drop.xs("activity", axis=1)
    & ~(agreed_bio_data_nan_drop.xs("model_simulation_0.05", axis=1))
    ][["bigg","chemical", "moa", "exchange", "model_simulation_0.05"]]

false_negatives

false_negatives.to_csv(os.getcwd() + "/" + 'Salb_false_negative.csv', index=True)


False negative (actual true, predicted false):



# 2. Try Gapfilling Function on false negatives

In [12]:
def basic_gapfill(bigg, model, reference, objective='growth', iter=1):
    """
    gapfill a model using reference model and cplex solver.
    
    Parameters:
    -----------
    bigg: String
    model: Model
    reference: Model
    """
    print(f"\nGapfilling {bigg} with: ")
    with reference:
        reference.solver = 'cplex'
        model.solver = 'cplex'
        model.objective = objective
        solution = gapfill(model, reference, demand_reactions=False, iterations=iter)
        print(solution)
    return solution 

def gapfill_medium(model, reference, bigg, type, exchange):
    """
    This function does gapfilling based on a chemical in the certain biolog medium and add an exchange reaction if needed"
    
    Parameters:
    -----------
    bigg (str) 
    category (str)
    exchange (str)
    """

    # when model has the exchange reaction, no need to add a reaction before gapfilling.  
    try:
        with model:
            medium = model.medium
            medium[exchange] = 0.8
            if type.startswith("C"):
                medium["EX_glc__D_e"] = 0

            elif type.startswith("N"):
                medium["EX_nh4_e"] = 0

            elif type.startswith("P"):
                medium["EX_pi_e"] = 0
            
            elif type.startswith("S"):
                medium["EX_so4_e"] = 0

            model.medium = medium
            model.reactions.EX_co2_e.bounds= (0, 1000)

            return basic_gapfill(bigg, model, reference, 'growth', 4)
    except:
        print(f"Gapfilling of the compound {bigg} - {type} - {exchange} is failed\n")
        return None


In [13]:
_model = model.copy()
with open('gapfill_reactions.txt', 'w') as file:
    for ind in false_negatives.index:
        file.write(f"Gapfilling {false_negatives['bigg'][ind]}, {false_negatives['moa'][ind]} with:\n")
        solution = gapfill_medium(_model, sco_model, false_negatives['bigg'][ind], false_negatives['moa'][ind], false_negatives['exchange'][ind])
        if solution:
            unique_solution = set([reaction for reaction_set in solution for reaction in reaction_set])
            reactions_str = ', '.join(str(reaction.id) for reaction in unique_solution)
            print(reactions_str)
            file.write(reactions_str + '\n\n')
        else:
            file.write('\n\n')

Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmplopuhu5q.lp
Reading time = 0.02 seconds
: 1874 rows, 4466 columns, 19614 nonzeros

Gapfilling arab__L with: 
[[<Reaction ARAI at 0x7f7812414c40>, <Reaction DTDP26DGR at 0x7f77e495f820>], [<Reaction ARAI at 0x7f7812414c40>, <Reaction DTDP26DGR at 0x7f77e495f820>], [<Reaction ARAI at 0x7f7812414c40>, <Reaction DTDP26DGR at 0x7f77e495f820>], [<Reaction ARAI at 0x7f7812414c40>, <Reaction DTDP26DGR at 0x7f77e495f820>]]
DTDP26DGR, ARAI

Gapfilling man with: 
[[<Reaction MANpts at 0x7f77f48791f0>], [<Reaction HEX4 at 0x7f77f48437c0>], [<Reaction HEX4 at 0x7f77f48437c0>], [<Reaction MANpts at 0x7f77f48791f0>]]
HEX4, MANpts

Gapfilling lyx__L with: 
[[<Reaction DTDP26DGR at 0x7f7812d977f0>, <Reaction FBA4 at 0x7f77f56393a0>, <Reaction KHK2 at 0x7f7813182cd0>, <Reaction LYXI at 0x7f7812e4f250>], [<Reaction DTDP26DGR at 0x7f7812d977f0>, <Reaction FBA4 at 0x7f77f56393a0>, <Reaction KHK2 at 0x7f7813182cd0>, <Reaction

In [14]:
# Initialize an empty list to store the extracted lists
gapfill_lists = []

# Open and read the file
with open('gapfill_reactions.txt', 'r') as file:
    lines = file.readlines()

# Process every second line and split it into a list
for i in range(1, len(lines), 3):
    reaction_list = [item.strip() for item in lines[i].strip().split(',')]
    gapfill_lists.extend(reaction_list)

merged_lists = set(gapfill_lists)
merged_lists.discard('')

# Print the extracted lists
print(merged_lists)

{'DTDP26DGR', 'ACONIs', 'GERDPM', 'PPRDNDH', 'DM_2mborn_c', 'CITRLt', 'HEX4', 'DAPAT', 'EX_h2s_e', 'FBA4', 'APTNAT', '5DGLCNR', 'EX_citr__L_e', 'MACACI', '2METBOD', 'ETHAAL', 'ARAI', 'PHYCBOXL', 'H2St1', 'MANpts', 'PPYRDC', 'LYXI', 'KHK2', 'NO2t2r'}


# 3. Get a short list by testing new reactions

In [15]:
short_list = []
for reaction_id in merged_lists:
    # Get the reaction from the reference model
    reaction = sco_model.reactions.get_by_id(reaction_id)
    model_temp = model.copy()
    
    with model_temp:
        # Add a copy of the reaction to your model
        model_temp.add_reactions([reaction.copy()])
        model_temp.solver ='cplex'
        model_temp.reactions.EX_co2_e.bounds= (0, 1000)

        solution = model_temp.optimize()
        print (f"------\nTest with {reaction_id} \nSolution: {solution}\n--------")

        if solution.objective_value < 1:
            short_list.append(reaction_id)

short_list

Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmpz_1geyxz.lp
Reading time = 0.02 seconds
: 1874 rows, 4466 columns, 19614 nonzeros


------
Test with DTDP26DGR 
Solution: <Solution 0.156 at 0x7f7764778e50>
--------
Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmpnctgqbsk.lp
Reading time = 0.01 seconds
: 1874 rows, 4466 columns, 19614 nonzeros
------
Test with ACONIs 
Solution: <Solution 0.052 at 0x7f77e3858bb0>
--------
Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmpfhsm9w3i.lp
Reading time = 0.01 seconds
: 1874 rows, 4466 columns, 19614 nonzeros
------
Test with GERDPM 
Solution: <Solution 0.052 at 0x7f77830a4160>
--------
Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmpruxexr31.lp
Reading time = 0.01 seconds
: 1874 rows, 4466 columns, 19614 nonzeros
------
Test with PPRDNDH 
Solution: <Solution 0.052 at 0x7f779d71a5e0>
--------
Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmp7e6w1a3m.lp
Reading time = 0.02 seconds
: 1874 rows, 4466 columns, 19614 nonzeros
------
Test with DM_

['DTDP26DGR',
 'ACONIs',
 'GERDPM',
 'PPRDNDH',
 'DM_2mborn_c',
 'CITRLt',
 'HEX4',
 'DAPAT',
 'EX_h2s_e',
 'FBA4',
 'APTNAT',
 '5DGLCNR',
 'EX_citr__L_e',
 'MACACI',
 '2METBOD',
 'ETHAAL',
 'ARAI',
 'PHYCBOXL',
 'H2St1',
 'MANpts',
 'PPYRDC',
 'LYXI',
 'KHK2',
 'NO2t2r']

In [16]:
import requests, re

def check_annotation_uniprot(reaction, taxonomy_id):
    ec_numbers = reaction.annotation.get('ec-code', [])
    result = []
    for ec in ec_numbers:
        # Ensure the EC number has only digits and dots
        if re.match(r'^\d+(\.\d+)+$', ec):
            print(ec)
            query = f'(taxonomy_id:{taxonomy_id}) AND (ec:{ec})'
            url = 'https://rest.uniprot.org/uniprotkb/search?'
            params = {
                'query': query,
                'format': 'list',
            }
            response = requests.get(url, params=params)
            if response.text != '':
                result.append(response.text)
    print(result)
    return result

def check_transport_reaction(reaction):
    compartments = {metabolite.compartment for metabolite in reaction.metabolites}
    is_transport = len(compartments) > 1
    
    print(f"transport reaction: {is_transport}")
   
    return is_transport

In [17]:
# Fact checking using different methods, if a reaction is not found in uniprot, then it's likely the reaction doesn't exist
taxonomy_id = 1886

final_list = []
for r in short_list:
    reaction = sco_model.reactions.get_by_id(r)
    print(f'--------\n{reaction}')
    if check_transport_reaction(reaction) == False:
        if reaction.annotation:
            if check_annotation_uniprot(reaction, taxonomy_id):
                final_list.append(r)

final_list  

--------
HEX4: atp_c + man_c --> adp_c + h_c + man6p_c
transport reaction: False
[]
--------
FBA4: xu1p__D_c <=> dhap_c + gcald_c
transport reaction: False
[]
--------
PPRDNDH: 2.0 h2o_c + nad_c + pprdn_c --> 5aptn_c + h_c + nadh_c
transport reaction: False
[]
--------
2METBOD: 2mygydpp_c + h2o_c --> 2mborn_c + ppi_c
transport reaction: False
[]
--------
NO2t2r: h_e + no2_e <=> h_c + no2_c
transport reaction: True
--------
5DGLCNR: 5dglcn_c + h_c + nadph_c <=> glcn_c + nadp_c
transport reaction: False
[]
--------
GERDPM: amet_c + grdp_c --> 2mygydpp_c + ahcys_c + h_c
transport reaction: False
[]
--------
EX_h2s_e: h2s_e --> 
transport reaction: False
[]
--------
MACACI: 4mlacac_c <=> 4fumacac_c
transport reaction: False
[]
--------
H2St1: h2s_e <=> h2s_c
transport reaction: True
--------
ACONIs: acon_T_c <=> acon_C_c
transport reaction: False
[]
--------
APTNAT: 5aptn_c + akg_c <=> glu__L_c + oxptn_c
transport reaction: False
[]
--------
MANpts: man_e + pep_c --> man6p_c + pyr_c
transp

[]

In [18]:
# Test added reactions
for reaction_id in short_list:
    # Get the reaction from the sco_model model
    reaction = sco_model.reactions.get_by_id(reaction_id)
    
    # Add a copy of the reaction to your model
    model.add_reactions([reaction.copy()])

In [19]:
def optimize_medium(bigg, type, exchange):
    """
    This function does optimize based on a chemical in the certain biolog medium and add an exchange reaction if needed
    """


    # when model has the exchange reaction, no need to add a reaction before gapfilling.    
    with model:
        medium = model.medium
        medium[exchange] = 0.8
        if type.startswith("C"):
            medium["EX_glc__D_e"] = 0

        elif type.startswith("N"):
            medium["EX_nh4_e"] = 0

        elif type.startswith("P"):
            medium["EX_pi_e"] = 0
        
        elif type.startswith("S"):
            medium["EX_so4_e"] = 0

        print(f"\n Optimize {bigg}, {type} with: ")
        model.medium = medium
        print(model.medium)
        model.solver = 'cplex'
        model.objective = 'growth'
        solution = model.optimize()
        print(solution.objective_value)
    return solution 
            
            
for ind in false_negatives.index:
    optimize_medium(false_negatives['bigg'][ind], false_negatives['moa'][ind], false_negatives['exchange'][ind])




 Optimize arab__L, C-Source, carbohydrate with: 
{'EX_arab__L_e': 0.8, 'EX_ca2_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 1000.0, 'EX_k_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_ni2_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_tungs_e': 1000.0, 'EX_zn2_e': 1000.0}
0.13031365898552

 Optimize man, C-Source, carbohydrate with: 
{'EX_ca2_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 1000.0, 'EX_k_e': 1000.0, 'EX_man_e': 0.8, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_ni2_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_tungs_e': 1000.0, 'EX_zn2_e': 1000.0}
0.15637639078262328

 Optimize lyx__L, C-Source

# 4. Gapfill Untestables with Sco model

In [20]:
def add_exchange(model, reference, metabolite_bigg, metabolite_name):
    """
    This function aims to add an extra exchange reaction based on metabolite to the target model using reference model.
    
    Parameters:
    -----------
    metabolite_bigg: String
    metabolite_name: String
    model: Cobra Model
    reference: Cobra Model
    """
    try:
        ex_rxn_id = f"EX_{metabolite_bigg}_e"
        ex_met_id = f"{metabolite_bigg}_e"
        cy_met_id = f"{metabolite_bigg}_c"
        print(f"Try to add {ex_rxn_id} reaction")
        if any(r.id == ex_rxn_id for r in model.reactions.query(ex_rxn_id)):
            if model.reactions.get_by_id(ex_rxn_id).boundary == False:
                model.add_boundary(model.metabolites.get_by_id(ex_met_id), type="exchange")
            return model, ex_rxn_id

        else:
            # Check if the reaction is already in reference model, 
            # if yes, just copy reaction to model,
            # if not, add a new reaction.
            
                if reference.reactions.query(ex_rxn_id) != []:
                    ex_rxn = reference.reactions.get_by_id(ex_rxn_id)
                    model.add_reactions([ex_rxn.copy()])
                    print (f"Exchange: {ex_rxn_id} is added")
                
                else:
                    # Check if the metabolites is in target model in both cytosol and external
                    # if yes, create new reaction based on reference model
                    # if not, create new metabolite
                    if any(m.id == cy_met_id for m in reference.metabolites.query(cy_met_id)):
                        ex_rxn = Reaction(id=ex_rxn_id, name=f"{metabolite_name}-exchange")
                        model.add_reactions([ex_rxn])
                        met = reference.metabolites.get_by_id(cy_met_id).copy()
                        met.id = ex_met_id
                        met.compartment = 'e'
                        ex_rxn.add_metabolites({met: -1})                        
                        print(f"Exchange: {ex_rxn_id} is added")

                    elif any(m.id == ex_met_id for m in reference.metabolites.query(ex_met_id)):
                        ex_rxn = Reaction(id=ex_rxn_id, name=f"{metabolite_name}-exchange")
                        model.add_reactions([ex_rxn])
                        met = reference.metabolites.get_by_id(ex_met_id).copy()
                        ex_rxn.add_metabolites({met: -1})                        
                        print(f"Exchange: {ex_rxn_id} is added")
                    else:
                        print(f'{metabolite_bigg} is not in reference model\n')
                        return model, None
                # Final check to see if the boundary is added
                if model.reactions.get_by_id(ex_rxn_id).boundary == False:
                    model.add_boundary(model.metabolites.get_by_id(met.id), type="exchange")
                return model, ex_rxn_id
    except:
        return model, None

In [21]:
# All Untestable Growth Conditions ()
print(f"\nUntestable conditions:\n")

# Mapping of bigg and biolog model
untestables = pd.read_csv(f"{os.getcwd()}/../untestable_metabolites.csv", converters={'Ignore': lambda x: True if x == 'TRUE' else False})

# Join two tables together
untestables = untestables.merge(agreed_bio_data[['activity', 'index', 'moa', 'bigg']], on='bigg', how='left')
untestables['ignore'] = untestables['ignore'].replace({None: False, 'FALSE': True})
untestables = untestables[["bigg","name", "moa", "ignore"]]
# untestables.to_csv('untestables.csv')
untestables


Untestable conditions:



,bigg,name,moa,ignore
0,2hbut,2 Hydroxybutyrate C4H7O3,"C-Source, carboxylic acid",False
1,2obut,2-Oxobutanoate,"C-Source, carboxylic acid",False
2,2pg,D-Glycerate 2-phosphate,"P-Source, organic",False
3,2pglyc,2-Phosphoglycolate,"P-Source, organic",False
4,35cgmp,"3',5'-Cyclic GMP","P-Source, organic",False
...,...,...,...,...
77,tym,Tyramine,"N-Source, other",False
78,tyrp,Phosphotyrosine,"P-Source, organic",False
79,tyrp,Phosphotyrosine,"P-Source, organic",False
80,urate,Urate C5H4N4O3,"N-Source, other",False


In [22]:
_model = model.copy()
with open('gapfill_reactions_sco_untestable.txt', 'w') as file:
    for ind in untestables.index:
        bigg = untestables['bigg'][ind]
        type = untestables['moa'][ind]
        name = untestables['name'][ind]

        # Only test ones that needs to be added
        if untestables['ignore'][ind] == False:
            file.write(f"Gapfilling {bigg}, {type} with:\n")
            print(f"---------------\nGapfilling {bigg}, {type} test:\n")
            # Try to add exchange reactions before gapfilling
            _model, ex_rxn = add_exchange(_model, sco_model, bigg, name)
            solution = None
            if ex_rxn:
                solution = gapfill_medium(_model, sco_model, bigg, type, ex_rxn)

            # Write gapfilling results
            if solution:
                unique_solution = set([reaction for reaction_set in solution for reaction in reaction_set])
                reactions_str = ', '.join(str(reaction.id) for reaction in unique_solution)
                file.write(reactions_str + '\n\n')
            else:
                file.write('\n\n')

Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmpwb_3z1u2.lp
Reading time = 0.01 seconds
: 1881 rows, 4514 columns, 19772 nonzeros
---------------
Gapfilling 2hbut, C-Source, carboxylic acid test:

Try to add EX_2hbut_e reaction
2hbut is not in reference model

---------------
Gapfilling 2obut, C-Source, carboxylic acid test:

Try to add EX_2obut_e reaction
Exchange: EX_2obut_e is added

Gapfilling 2obut with: 
Gapfilling of the compound 2obut - C-Source, carboxylic acid - EX_2obut_e is failed

---------------
Gapfilling 2pg, P-Source, organic test:

Try to add EX_2pg_e reaction
Exchange: EX_2pg_e is added

Gapfilling 2pg with: 
Gapfilling of the compound 2pg - P-Source, organic - EX_2pg_e is failed

---------------
Gapfilling 2pglyc, P-Source, organic test:

Try to add EX_2pglyc_e reaction
Exchange: EX_2pglyc_e is added

Gapfilling 2pglyc with: 
Gapfilling of the compound 2pglyc - P-Source, organic - EX_2pglyc_e is failed

---------------
Gapfilling 3

In [23]:
# Initialize an empty list to store the extracted lists
gapfill_lists = []

# Open and read the file
with open('gapfill_reactions_uni_untestable.txt', 'r') as file:
    lines = file.readlines()

# Process every second line and split it into a list
for i in range(1, len(lines), 3):
    reaction_list = [item.strip() for item in lines[i].strip().split(',')]
    gapfill_lists.extend(reaction_list)

merged_lists = set(gapfill_lists)
merged_lists.discard('')

# Print the extracted lists
print(merged_lists)

{'ALLTNt2r', 'GAMpts', 'SALCpts', 'ARGAGMt7', 'ACGApts'}


In [24]:
for reaction_id in merged_lists:
    # Get the reaction from the sco_model model
    reaction = sco_model.reactions.get_by_id(reaction_id)
    
    # Add a copy of the reaction to your model
    model.add_reactions([reaction.copy()])


# 5. Compare model before and after

In [3]:
# Compare two models by reaction numbers and metabolites.
updated_model = read_sbml_model(f"{os.getcwd()}/Salb-GEM-Sco-gapfill.xml")



'' is not a valid SBML 'SId'.


In [6]:
model

Name,
Memory address,7fa688a85850
Number of metabolites,1874
Number of reactions,2233
Number of genes,1386
Number of groups,0
Objective expression,1.0*growth - 1.0*growth_reverse_5ff58
Compartments,"Cytoplasm, Extracellular"


In [4]:
updated_model

Name,
Memory address,7fa688a91640
Number of metabolites,1886
Number of reactions,2262
Number of genes,1405
Number of groups,0
Objective expression,1.0*growth - 1.0*growth_reverse_5ff58
Compartments,"Cytoplasm, Extracellular"


In [5]:
sco_model

Name,Sco_GEM_v1_3_1
Memory address,7fa6a9781e20
Number of metabolites,2064
Number of reactions,2599
Number of genes,1775
Number of groups,7
Objective expression,1.0*BIOMASS_SCO_tRNA - 1.0*BIOMASS_SCO_tRNA_reverse_d3620
Compartments,"Cytoplasm, Extracellular"


In [25]:
model.reactions.EX_co2_e.bounds= (0, 1000)

cobra.io.write_sbml_model(model, 'Salb-GEM-Sco-gapfill.xml')